In [1]:
import findspark
findspark.init('/opt/spark')

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('lr_app').getOrCreate()

22/05/25 09:40:40 WARN Utils: Your hostname, ls-Aspire-A515-51G resolves to a loopback address: 127.0.1.1; using 192.168.15.8 instead (on interface wlp3s0)
22/05/25 09:40:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/25 09:40:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/25 09:40:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
df = spark.read.csv('titanic.csv', header=True, inferSchema=True)

In [5]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [6]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [7]:
cols = df.select(['Survived',
                 'Pclass',
                 'Sex',
                 'Age',
                 'SibSp',
                 'Parch',
                 'Fare',
                 'Embarked'])

In [8]:
final_df = cols.na.drop()

In [9]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer

In [10]:
gender_indexer = StringIndexer(inputCol='Sex', outputCol='SexIndex') # Assign a number for each cat variable
# A B C : step 1
# 0 1 2: step 2
# One Hot Encoder: step 3
# Key A B C
# Example A
# [1, 0, 0]
gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol='SexVec')


In [11]:
embark_indexer = StringIndexer(inputCol='Embarked', outputCol='EmbarkIndex') # Assign a number for each cat variable
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex', outputCol='EmbarkVec')


In [12]:
assembler = VectorAssembler(inputCols=['Pclass', 'SexVec', 'EmbarkVec', 'Age', 'SibSp', 'Parch', 'Fare'],
                           outputCol='features')

In [13]:
from pyspark.ml.classification import LogisticRegression

In [14]:
from pyspark.ml import Pipeline

In [15]:
log_reg = LogisticRegression(featuresCol='features', labelCol='Survived')

In [16]:
pipeline = Pipeline(stages=[gender_indexer, embark_indexer, 
                           gender_encoder, embark_encoder,
                           assembler, log_reg])

In [17]:
train_data, test_data = final_df.randomSplit([0.7, 0.3])

In [18]:
fit_model = pipeline.fit(train_data)

In [19]:
results = fit_model.transform(test_data)

In [22]:
results.printSchema()

root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- SexIndex: double (nullable = false)
 |-- EmbarkIndex: double (nullable = false)
 |-- SexVec: vector (nullable = true)
 |-- EmbarkVec: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [20]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [23]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Survived')

In [25]:
results.select('Survived', 'prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [26]:
AUC = evaluator.evaluate(results)

In [27]:
AUC

0.783957033957034